In [39]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np

from collections import defaultdict
from tqdm import tqdm 
from heapq import nlargest

tqdm.pandas()

In [40]:
df_pred = pd.read_csv("../Data/df_pred_ext.csv").drop("Unnamed: 0", axis=1)

In [41]:
df_pred = df_pred[df_pred["source"] == 0]

In [42]:
career_paths = df_pred.groupby("candidate_id")["isco_code4"]

In [43]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values)

100%|██████████| 382133/382133 [00:10<00:00, 35514.46it/s]


In [44]:
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens > 1)]

In [45]:
career_paths = career_paths.loc[career_paths.apply(lambda x: x[-1] != x[-2])]
len(career_paths)

61293

In [46]:
career_paths_train = career_paths.iloc[:int(0.8 * 61293)]
career_paths_test = career_paths.iloc[int(0.8 * 61293):]

In [47]:
switches = defaultdict(lambda: defaultdict(int))

for c, jobs in career_paths_train.iteritems():
    for i in range(len(jobs)):
        if i < len(jobs) - 1 and jobs[i] != jobs[i + 1]:
            switches[jobs[i]][jobs[i  + 1]] += 1

In [83]:
most_common = pd.Series({k: set(nlargest(1, v, key=v.get)) for k, v in switches.items()})

In [84]:
most_common.head()

208    {348}
348    {208}
344    {345}
345    {344}
185    {177}
dtype: object

In [85]:
most_common_switch = career_paths_test.progress_apply(lambda x: x[-1] in most_common.loc[x[-2]] if x[-2] in most_common.index else False).mean()

100%|██████████| 12259/12259 [00:00<00:00, 43083.80it/s]


In [86]:
most_common_switch, 1.96 * np.sqrt(((1 - most_common_switch)* (most_common_switch)) / (113724*0.2))

(0.30573456236234603, 0.005987562386471285)

In [62]:
print(f"Most common switch accuracy: {most_common_switch * 100:>.2f}%")

Most common switch accuracy: 60.36%
